# Step0: Install environment

In [3]:
import vapoursynth as vs
import havsfunc as haf
import mvsfunc as mvf
from vapoursynth import core
import awsmfunc as awsf

# Step1: Select BDMV Largest M2TS and its MLPS

In [ ]:
x265_filename=r"C:\Users\heber\Music\How.to.Train.Your.Dragon.2.2014.2160p.BluRay.x265.DTS-louco@PTHOME\How.to.Train.Your.Dragon.2.2014.2160p.BluRay.x265.DTS-louco@PTHOME.mkv"

# Step2: BDMV to MKV

# Step3: Transcode

In [4]:
import os, pyperclip, re, subprocess
from pymediainfo import MediaInfo
from datetime import datetime
# from utils import *
crf=17.0
qcomp=0.7
aq_mode=3
aq_s=0.7
bitdepth=8
newline="\r\n"
remotecwd="~/compressor"
remotehost="amd"
PROCUCT_DEST_DIR=r"/home/qianyi/Downloads/"
MOVIE=r"/home/qianyi/Downloads/The.Boat.2018.1080p.FRA.Blu-ray.REMUX.DTS-louco@PTH.mkv"

## get remux capture and crop black borders

In [5]:
get_captures("REMUX", MOVIE)

NameError: name 'get_captures' is not defined

## H264: Test params

In [10]:
test=""
x264_stmt=f"x264 --demuxer y4m --preset slower --profile high --level 4.1 --ref 4 \
--crf {crf} --qcomp {qcomp} --aq-mode {aq_mode} --aq-strength {aq_s} --bframes 11 --me umh  --subme 11 \
--merange 48 --no-fast-pskip --no-dct-decimate --direct auto --psy-rd 1.00:0.00 \
--vbv-bufsize 78125 --vbv-maxrate 62500 --deblock -3:-3 --b-adapt 2 --keyint 240 \
--min-keyint 1 --no-mbtree --trellis 2 --chroma-qp-offset -1 --rc-lookahead 72 --output 264.mkv -"

x265_stmt=f"x265 --y4m -D 10 --profile main10 --level-idc 5.1 --high-tier --preset slower --crf {crf} --qcomp {qcomp} --aq-mode {aq_mode} \
--aq-strength {aq_s} --rd 4 --psy-rd 2.0 --psy-rdoq 1.0 --rdoq-level 2 --deblock -1:-1 --cbqpoffs -1 --crqpoffs -3 --ctu 32 \
--qg-size 8 --no-sao --no-sao-non-deblock --selective-sao 0 --tu-intra-depth 4 --tu-inter-depth 4 --no-opt-qp-pps \
--no-opt-ref-list-length-pps --aud --repeat-headers --me 3 --subme 5 --merange 48 --b-intra --limit-tu 0 --no-rect \
--no-amp --no-open-gop --pools + --keyint 240 --min-keyint 1 --bframes 8 --max-merge 4 --ref 4 --weightb --rc-lookahead 72 \
--scenecut 40 --no-strong-intra-smoothing --vbv-bufsize 160000 --vbv-maxrate 160000 --input-depth 10 --output 265.mkv -"

test="-ss 3900 -t 60"
crop="-filter:v \"crop=in_w:in_h-280\""

args=x264_stmt.replace(" --","\n --").split("\n")[1:-1]
args=[i.strip().replace("--","").replace(" ","=").replace(":",",") for i in args]
args5=x265_stmt.replace(" --","\n --").split("\n")[1:-1]
args5=[i.strip().replace("--","").replace(" ","=").replace(":",",") for i in args5]
# print("\n".join(["x264 参数："]+args))

remote_x264stmt=f'vspipe.exe --y4m "{os. getcwd()}\\vs.py" - |zstd -0|python {os. getcwd()}\\aio_buffer.py|\
ssh {remotehost} "python3 {remotecwd}/aio_buffer.py|zstdcat|{x264_stmt} 2>&1 | tee x264.log"{newline}'

print("==========\n远程执行编码 x264:")
cmd=f'set MOVIE={MOVIE}{newline}'
cmd+=f'set TEST=YES{newline}'
cmd+=remote_x264stmt
cmd+=f'scp {remotehost}:~/264.mkv {os. getcwd()}\\264.mkv{newline}'
# cmd+=f'scp {remotehost}:~/264.mkv D:\\BT\\samples\\264.mkv{newline}'
cmd+=f'pause{newline}'
pyperclip.copy(cmd)
print(cmd)

print("===FFMPEG===")
x264_filename="".join(re.findall(r'([^\\]*$)', MOVIE)).replace("REMUX", "x264")
x265_filename="".join(re.findall(r'([^\\]*$)', MOVIE)).replace("REMUX", "x265.10bit")

print(f"ffmpeg {test} -i {MOVIE} -c:v libx264 -profile:v high {crop} -preset slow -x264-params {':'.join(args)} \
-max_muxing_queue_size 1024 -c:a copy -c:s copy -max_muxing_queue_size 1024 -map 0 {x264_filename} 2>&1|tee x264.log" )
print(f"ffmpeg {test} -i {MOVIE} -c:v libx265 -profile:v main10 {crop} -preset slow -x265-params {':'.join(args5)} \
-max_muxing_queue_size 1024 -c:a copy -c:s copy -max_muxing_queue_size 1024 -map 0 {x265_filename} 2>&1|tee x265.log" )
print(f"echo DONE")

远程执行编码 x264:
set MOVIE=/home/qianyi/Downloads/The.Boat.2018.1080p.FRA.Blu-ray.REMUX.DTS-louco@PTH.mkv
set TEST=YES
vspipe.exe --y4m "/home/qianyi/Downloads/x264-265-batch-compressor\vs.py" - |zstd -0|python /home/qianyi/Downloads/x264-265-batch-compressor\aio_buffer.py|ssh amd "python3 ~/compressor/aio_buffer.py|zstdcat|x264 --demuxer y4m --preset slower --profile high --level 4.1 --ref 4 --crf 17.0 --qcomp 0.7 --aq-mode 3 --aq-strength 0.7 --bframes 11 --me umh  --subme 11 --merange 48 --no-fast-pskip --no-dct-decimate --direct auto --psy-rd 1.00:0.00 --vbv-bufsize 78125 --vbv-maxrate 62500 --deblock -3:-3 --b-adapt 2 --keyint 240 --min-keyint 1 --no-mbtree --trellis 2 --chroma-qp-offset -1 --rc-lookahead 72 --output 264.mkv - 2>&1 | tee x264.log"
scp amd:~/264.mkv /home/qianyi/Downloads/x264-265-batch-compressor\264.mkv
pause

===FFMPEG===
ffmpeg -ss 3900 -t 60 -i /home/qianyi/Downloads/The.Boat.2018.1080p.FRA.Blu-ray.REMUX.DTS-louco@PTH.mkv -c:v libx264 -profile:v high -filter:v "cr

## H264&H265

In [2]:
x264_stmt=f"x264 --demuxer y4m --preset slower --profile high --level 4.1 --ref 4 \
--crf {crf} --qcomp {qcomp} --aq-mode {aq_mode} --aq-strength {aq_s} --bframes 11 --me umh  --subme 11 \
--merange 48 --no-fast-pskip --no-dct-decimate --direct auto --psy-rd 1.00:0.00 \
--vbv-bufsize 78125 --vbv-maxrate 62500 --deblock -3:-3 --b-adapt 2 --keyint 240 \
--min-keyint 1 --no-mbtree --trellis 2 --chroma-qp-offset -1 --rc-lookahead 72 --output 264.mkv -"

x265_stmt=f"x265 --y4m -D 10 --profile main10 --level-idc 5.1 --high-tier --preset slower --crf {crf} --qcomp {qcomp} --aq-mode {aq_mode} \
--aq-strength {aq_s} --rd 4 --psy-rd 2.0 --psy-rdoq 1.0 --rdoq-level 2 --deblock -1:-1 --cbqpoffs -1 --crqpoffs -3 --ctu 32 \
--qg-size 8 --no-sao --no-sao-non-deblock --selective-sao 0 --tu-intra-depth 4 --tu-inter-depth 4 --no-opt-qp-pps \
--no-opt-ref-list-length-pps --aud --repeat-headers --me 3 --subme 5 --merange 48 --b-intra --limit-tu 0 --no-rect \
--no-amp --no-open-gop --pools + --keyint 240 --min-keyint 1 --bframes 8 --max-merge 4 --ref 4 --weightb --rc-lookahead 72 \
--scenecut 40 --no-strong-intra-smoothing --vbv-bufsize 160000 --vbv-maxrate 160000 --input-depth 10 --output 265.mkv -"


remote_stmt=f'vspipe.exe --y4m "{os. getcwd()}\\vs.py" - |zstd -0 -T4|python {os. getcwd()}\\aio_buffer.py|\
ssh {remotehost} "python3 {remotecwd}/aio_buffer.py|zstdcat| tee \
>({x264_stmt} 2>&1 | tee x264.log) \
>({x265_stmt} 2>&1 | tee x265.log) \
>/dev/null"{newline}'


print("==========\n远程执行编码 x264+x265:")
cmd=f'set MOVIE={MOVIE}{newline}'
cmd=f'set TEST=NO{newline}'
cmd+=remote_stmt
cmd+=f'scp {remotehost}:~/264.mkv {os. getcwd()}\\264.mkv{newline}'
cmd+=f'scp {remotehost}:~/265.mkv {os. getcwd()}\\265.mkv{newline}'

cmd+=f'ssh {remotehost} "cat x264.log|sed \\"s/.*\\r//g\\" " >{os. getcwd()}\\x264.log & '
cmd+=f'ssh {remotehost} "cat x265.log|sed \\"s/.*\\r//g\\" " >{os. getcwd()}\\x265.log{newline}'
cmd+=f'pause{newline}'
pyperclip.copy(cmd)
print(cmd)

远程执行编码 x264+x265:
set TEST=NO
vspipe.exe --y4m "C:\Users\heber\Documents\Compressor\vs.py" - |zstd -0 -T4|python C:\Users\heber\Documents\Compressor\aio_buffer.py|ssh amd "python3 ~/compressor/aio_buffer.py|zstdcat| tee >(x264 --demuxer y4m --preset slower --profile high --level 4.1 --ref 4 --crf 19.0 --qcomp 0.7 --aq-mode 3 --aq-strength 0.7 --bframes 11 --me umh  --subme 11 --merange 48 --no-fast-pskip --no-dct-decimate --direct auto --psy-rd 1.00:0.00 --vbv-bufsize 78125 --vbv-maxrate 62500 --deblock -3:-3 --b-adapt 2 --keyint 240 --min-keyint 1 --no-mbtree --trellis 2 --chroma-qp-offset -1 --rc-lookahead 72 --output 264.mkv - 2>&1 | tee x264.log) >(x265 --y4m -D 10 --profile main10 --level-idc 5.1 --high-tier --preset slower --crf 19.0 --qcomp 0.7 --aq-mode 3 --aq-strength 0.7 --rd 4 --psy-rd 2.0 --psy-rdoq 1.0 --rdoq-level 2 --deblock -1:-1 --cbqpoffs -1 --crqpoffs -3 --ctu 32 --qg-size 8 --no-sao --no-sao-non-deblock --selective-sao 0 --tu-intra-depth 4 --tu-inter-depth 4 --no-op

# H265 4K

In [2]:
x265_stmt=f'x265 --y4m -D 10 --profile main10 --level-idc 5.1 --high-tier --preset slower \
--crf {crf} --qcomp {qcomp} --aq-mode {aq_mode} \
--aq-strength {aq_s} --psy-rdoq 1 --psy-rd 2  --bframes 11 \
--ctu 64 --min-cu-size 8 --limit-tu 0 --tu-intra-depth 4 --tu-inter-depth 4 \
--me 3 --subme 6 --merange 48 --analyze-src-pics \
--max-merge 5 --early-skip --b-intra \
--no-open-gop --min-keyint 1 --keyint 240 --ref 6 --fades --b-adapt 2 \
--no-strong-intra-smoothing --cbqpoffs -1 --crqpoffs -3 \
--rdoq-level 2 --nr-inter 20 --nr-intra 10 \
--qg-size 32 \
--rd 4 --no-limit-modes --limit-refs 1 --no-rect --rc-lookahead 72 --tskip-fast \
--rdpenalty 0 --no-splitrd-skip \
--deblock -3:-3 --nr-inter 20 --nr-intra 10 \
--no-sao --no-sao-non-deblock --selective-sao 0 \
--no-opt-qp-pps --no-opt-ref-list-length-pps --repeat-headers \
--no-temporal-layers --no-amp --pools + \
--vbv-bufsize 160000 --vbv-maxrate 150000 --lookahead-slices 5 --input-depth 10 \
--hdr10 --hdr10-opt --colormatrix bt2020nc --colorprim bt2020 --transfer smpte2084 \
--master-display \'G(13250,34500)B(7500,3000)R(34000,16000)WP(15635,16450)L(10000000,0)\' --output 265.mkv -'
# --max-cll 922,239 
'''
BT.2020 	G(8500,39850)B(6550,2300)R(35400,14600)WP(15635,16450)L(40000000,50)
Display P3    G(13250,34500)B(7500,3000)R(34000,16000)WP(15635,16450)L(40000000,50)
'''

# with open("x265.sh","w") as f:
#     f.write(f'zstdcat ~/yrm/src|{x265_stmt} 2>&1 | tee x265.log')
# remote_x265stmt=f'vspipe.exe --y4m "{os. getcwd()}\\vs.py" - |zstd -0 -T8|ssh {remotehost} "cat>~/y4m/src"'
# remote_convertcmd=f'scp {os.getcwd()}\\x265.sh {remotehost}:{remotecwd}'
    
remote_stmt=f'vspipe.exe --y4m "{os. getcwd()}\\vs.py" - |zstd -0 -T4|python {os. getcwd()}\\aio_buffer.py|\
ssh {remotehost} "python3 {remotecwd}/aio_buffer.py|zstdcat| tee \
>({x265_stmt} 2>&1 | tee x265.log) \
>/dev/null"{newline}'


print("==========\n远程执行编码 x265:")
cmd=f'set MOVIE={MOVIE}{newline}'
cmd+=f'set TEST=NO{newline}'
cmd+=remote_stmt
cmd+=f'scp {remotehost}:~/265.mkv {os. getcwd()}\\265.mkv{newline}'

cmd+=f'ssh {remotehost} "cat x265.log|sed \'s/.*\\r//g\' " >{os. getcwd()}\\x265.log{newline}'

pyperclip.copy(cmd)
print(cmd)

远程执行编码 x265:
set MOVIE=C:\Users\heber\Music\Get.Out.2017.UHD.Blu-Ray.2160p.REMUX.DTS-louco@PTH.mkv
set TEST=NO
vspipe.exe --y4m "C:\Users\heber\Documents\Compressor\vs.py" - |zstd -0 -T4|python C:\Users\heber\Documents\Compressor\aio_buffer.py|ssh amd "python3 ~/compressor/aio_buffer.py|zstdcat| tee >(x265 --y4m -D 10 --profile main10 --level-idc 5.1 --high-tier --preset slower --crf 18.0 --qcomp 0.7 --aq-mode 3 --aq-strength 0.7 --psy-rdoq 1 --psy-rd 2  --bframes 11 --ctu 64 --min-cu-size 8 --limit-tu 0 --tu-intra-depth 4 --tu-inter-depth 4 --me 3 --subme 6 --merange 48 --analyze-src-pics --max-merge 5 --early-skip --b-intra --no-open-gop --min-keyint 1 --keyint 240 --ref 6 --fades --b-adapt 2 --no-strong-intra-smoothing --cbqpoffs -1 --crqpoffs -3 --rdoq-level 2 --nr-inter 20 --nr-intra 10 --qg-size 32 --rd 4 --no-limit-modes --limit-refs 1 --no-rect --rc-lookahead 72 --tskip-fast --rdpenalty 0 --no-splitrd-skip --deblock -3:-3 --nr-inter 20 --nr-intra 10 --no-sao --no-sao-non-debloc

## H264 & H265 offline

In [5]:
x264_stmt=f"x264 --demuxer y4m --preset slower --profile high --level 4.1 --ref 4 \
--crf {crf} --qcomp {qcomp} --aq-mode {aq_mode} --aq-strength {aq_s} --bframes 11 --me umh  --subme 11 \
--merange 48 --no-fast-pskip --no-dct-decimate --direct auto --psy-rd 1.00:0.00 \
--vbv-bufsize 78125 --vbv-maxrate 62500 --deblock -3:-3 --b-adapt 2 --keyint 240 \
--min-keyint 1 --no-mbtree --trellis 2 --chroma-qp-offset -1 --rc-lookahead 72 --output 264.mkv -"

x265_stmt=f"x265 --y4m -D 10 --profile main10 --level-idc 5.1 --high-tier --preset slower --crf {crf} --qcomp {qcomp} --aq-mode {aq_mode} \
--aq-strength {aq_s} --rd 4 --psy-rd 2.0 --psy-rdoq 1.0 --rdoq-level 2 --deblock -1:-1 --cbqpoffs -1 --crqpoffs -3 --ctu 32 \
--qg-size 8 --no-sao --no-sao-non-deblock --selective-sao 0 --tu-intra-depth 4 --tu-inter-depth 4 --no-opt-qp-pps \
--no-opt-ref-list-length-pps --aud --repeat-headers --me 3 --subme 5 --merange 48 --b-intra --limit-tu 0 --no-rect \
--no-amp --no-open-gop --pools + --keyint 240 --min-keyint 1 --bframes 8 --max-merge 4 --ref 4 --weightb --rc-lookahead 72 \
--scenecut 40 --no-strong-intra-smoothing --vbv-bufsize 160000 --vbv-maxrate 160000 --input-depth 10 --output 265.mkv -"

remote_stmt=f'zstdcat ~/yrm/src| tee \
>({x264_stmt} 2>&1 | tee x264.log) \
>({x265_stmt} 2>&1 | tee x265.log) \
>/dev/null{newline}'

with open("x26x.sh","w") as f:
    f.write(remote_stmt)
remote_x265stmt=f'vspipe.exe --y4m "{os. getcwd()}\\vs.py" - |zstd -0 -T8|ssh {remotehost} "cat>~/y4m/src"'
remote_convertcmd=f'scp {os.getcwd()}\\x26x.sh {remotehost}:{remotecwd}'

print("==========\n远程执行编码 x265:")
cmd=f'set MOVIE={MOVIE}{newline}'
cmd+=f'set TEST=NO{newline}'
cmd+=remote_x265stmt+newline
cmd+=remote_convertcmd+newline
cmd+=f'pause{newline}'
cmd+=f'scp {remotehost}:{remotecwd}/264.mkv {os. getcwd()}\\264.mkv{newline}'
cmd+=f'scp {remotehost}:{remotecwd}/265.mkv {os. getcwd()}\\265.mkv{newline}'

pyperclip.copy(cmd)
print(cmd)

远程执行编码 x265:
set MOVIE=C:\Users\heber\Music\How.to.Train.Your.Dragon.2.2014.2160p.BluRay.REMUX.HEVC.DTS-MA.7.1-PTHOME\How.to.Train.Your.Dragon.2.2014.2160p.BluRay.REMUX.HEVC.DTS-MA.7.1-PTHOME.mkv
set TEST=NO
vspipe.exe --y4m "C:\Users\heber\Documents\Compressor\vs.py" - |zstd -0 -T8|ssh ubuntu20local "cat>~/y4m/src"
scp C:\Users\heber\Documents\Compressor\x26x.sh ubuntu20local:~/compressor
pause
scp ubuntu20local:~/compressor/265.mkv C:\Users\heber\Documents\Compressor\265.mkv



# Step4: Remux MKV
## x264

In [6]:
# title="".join(re.findall(r'([^\\]*\d{4}p)', MOVIE))
# NEWMOVIE=f'{os.getcwd()}\\{title}.BluRay.x264.DTS-PTH.mkv'
x264_filename="".join(re.findall(r'([^\\]*$)', MOVIE)).replace("REMUX", "x264")
x264_foldername=PROCUCT_DEST_DIR+"\\"+x264_filename.replace(".mkv",'')
x264_fullpath=f'{x264_foldername}\\{x264_filename}'
print(x264_filename)
cmd=f'mkdir {x264_foldername} ^& '
cmd+=f'{os. getcwd()}\\mkvtoolnix\\mkvmerge.exe -o "{x264_fullpath}" -D ( "{MOVIE}" )\
    -A -S -T -M -B --no-chapters ( "{os. getcwd()}\\264.mkv" )'
pyperclip.copy(cmd)
print(f"即将执行: \n   [剩] {MOVIE}\n + [视] {os. getcwd()}\\264.mkv\n-> [全] {x264_fullpath}\n=================")
print(cmd)

Legendary.La.Rose.Noire.1992.Blu-ray.1080p.x264.TrueHD.5.1-louco@PTHome.mkv
即将执行: 
   [剩] Legendary.La.Rose.Noire.1992.Blu-ray.1080p.REMUX.TrueHD.5.1-louco@PTHome.mkv
 + [视] I:\Users\heber\Documents\Compressor\264.mkv
-> [全] I:\Users\heber\Music\Legendary.La.Rose.Noire.1992.Blu-ray.1080p.x264.TrueHD.5.1-louco@PTHome\Legendary.La.Rose.Noire.1992.Blu-ray.1080p.x264.TrueHD.5.1-louco@PTHome.mkv
mkdir I:\Users\heber\Music\Legendary.La.Rose.Noire.1992.Blu-ray.1080p.x264.TrueHD.5.1-louco@PTHome ^& I:\Users\heber\Documents\Compressor\mkvtoolnix\mkvmerge.exe -o "I:\Users\heber\Music\Legendary.La.Rose.Noire.1992.Blu-ray.1080p.x264.TrueHD.5.1-louco@PTHome\Legendary.La.Rose.Noire.1992.Blu-ray.1080p.x264.TrueHD.5.1-louco@PTHome.mkv" -D ( "Legendary.La.Rose.Noire.1992.Blu-ray.1080p.REMUX.TrueHD.5.1-louco@PTHome.mkv" )    -A -S -T -M -B --no-chapters ( "I:\Users\heber\Documents\Compressor\264.mkv" )


In [21]:
subprocess.run(f"start cmd /c {cmd} ^& pause", shell=True)
None

In [12]:
get_captures("x264", x264_filename)
generate_videoinfo(f'{x264_foldername}\\{x264_filename}',"x264.log")

Capture x264
[quote][font=Lucida Console]
.Release.Info
ENCODER...................: PTH
RELEASE NAME..............: Legendary.La.Rose.Noire.1992.Blu-ray.1080p.REMUX.TrueHD.5.1-loudo@PTHome
RELEASE DATE..............: 2021-01-25
RELEASE SiZE..............: 10.084 GB
SOURCE....................: 

.Media.Info
RUNTiME...................: 1 h 34 min
ViDEO CODEC...............: x264 High@L4.1 @ 11.97Mb/s
RESOLUTiON................: 1920x1080
DiSPLAY ASPECT RATiO......: 1.778
FRAME RATE................: 24.000
BIT DEPTH.................: 8 bits
AUDiO CODEC...............: Chinese AC-3 2 @ 384.0Kb/s 
AUDiO CODEC...............: Chinese MLP FBA 5.1 @ 2745.61Kb/s 
 
SUBTiTLES.................: S_HDMV/PGS : Chinese(chs)
SUBTiTLES.................: S_HDMV/PGS : Chinese(cht)
SUBTiTLES.................: S_HDMV/PGS : English(eng)


.x264.Info
[/quote]


'[quote][font=Lucida Console]\n.Release.Info\nENCODER...................: PTH\nRELEASE NAME..............: Legendary.La.Rose.Noire.1992.Blu-ray.1080p.REMUX.TrueHD.5.1-loudo@PTHome\nRELEASE DATE..............: 2021-01-25\nRELEASE SiZE..............: 10.084 GB\nSOURCE....................: \n\n.Media.Info\nRUNTiME...................: 1 h 34 min\nViDEO CODEC...............: x264 High@L4.1 @ 11.97Mb/s\nRESOLUTiON................: 1920x1080\nDiSPLAY ASPECT RATiO......: 1.778\nFRAME RATE................: 24.000\nBIT DEPTH.................: 8 bits\nAUDiO CODEC...............: Chinese AC-3 2 @ 384.0Kb/s \nAUDiO CODEC...............: Chinese MLP FBA 5.1 @ 2745.61Kb/s \n \nSUBTiTLES.................: S_HDMV/PGS : Chinese(chs)\nSUBTiTLES.................: S_HDMV/PGS : Chinese(cht)\nSUBTiTLES.................: S_HDMV/PGS : English(eng)\n\n\n.x264.Info\n[/quote]'

## x265

In [7]:
x265_filename="".join(re.findall(r'([^\\]*$)', MOVIE)).replace("REMUX", "x265.10bit")
x265_foldername=PROCUCT_DEST_DIR+"\\"+x265_filename.replace(".mkv",'')
NEWMOVIE=f'{x265_foldername}\\{x265_filename}'
x265_fullpath=NEWMOVIE
cmd=f'mkdir {x265_foldername} ^& '
cmd+=f'{os. getcwd()}\\mkvtoolnix\\mkvmerge.exe -o "{NEWMOVIE}" -D ( "{MOVIE}" )\
    -A -S -T -M -B --no-chapters ( "{os. getcwd()}\\265.mkv" )'
pyperclip.copy(cmd)
print(f"即将执行: \n   [剩] {MOVIE}\n + [视] {os. getcwd()}\\265.mkv\n-> [全] {NEWMOVIE}\n=================")
print(cmd)

即将执行: 
   [剩] Legendary.La.Rose.Noire.1992.Blu-ray.1080p.REMUX.TrueHD.5.1-louco@PTHome.mkv
 + [视] I:\Users\heber\Documents\Compressor\265.mkv
-> [全] I:\Users\heber\Music\Legendary.La.Rose.Noire.1992.Blu-ray.1080p.x265.10bit.TrueHD.5.1-louco@PTHome\Legendary.La.Rose.Noire.1992.Blu-ray.1080p.x265.10bit.TrueHD.5.1-louco@PTHome.mkv
mkdir I:\Users\heber\Music\Legendary.La.Rose.Noire.1992.Blu-ray.1080p.x265.10bit.TrueHD.5.1-louco@PTHome ^& I:\Users\heber\Documents\Compressor\mkvtoolnix\mkvmerge.exe -o "I:\Users\heber\Music\Legendary.La.Rose.Noire.1992.Blu-ray.1080p.x265.10bit.TrueHD.5.1-louco@PTHome\Legendary.La.Rose.Noire.1992.Blu-ray.1080p.x265.10bit.TrueHD.5.1-louco@PTHome.mkv" -D ( "Legendary.La.Rose.Noire.1992.Blu-ray.1080p.REMUX.TrueHD.5.1-louco@PTHome.mkv" )    -A -S -T -M -B --no-chapters ( "I:\Users\heber\Documents\Compressor\265.mkv" )


In [23]:
subprocess.run(f"start cmd /c {cmd} ^& pause", shell=True)

CompletedProcess(args='start cmd /c mkdir I:\\Users\\heber\\Music\\Legendary.La.Rose.Noire.1992.Blu-ray.1080p.x265.10bit.TrueHD.5.1-louco@PTHome ^& I:\\Users\\heber\\Documents\\Compressor\\mkvtoolnix\\mkvmerge.exe -o "I:\\Users\\heber\\Music\\Legendary.La.Rose.Noire.1992.Blu-ray.1080p.x265.10bit.TrueHD.5.1-louco@PTHome\\Legendary.La.Rose.Noire.1992.Blu-ray.1080p.x265.10bit.TrueHD.5.1-louco@PTHome.mkv" -D ( "Legendary.La.Rose.Noire.1992.Blu-ray.1080p.REMUX.TrueHD.5.1-louco@PTHome.mkv" )    -A -S -T -M -B --no-chapters ( "I:\\Users\\heber\\Documents\\Compressor\\265.mkv" ) ^& pause', returncode=0)

In [27]:
generate_videoinfo(x265_fullpath,"x265.log")
print(x265_filename)
get_captures("x265", x265_fullpath)

[quote][font=Lucida Console]
.Release.Info
ENCODER...................: PTH
RELEASE NAME..............: Forrest.Gump.1994.Remastered.1080p.Blu-ray.x265.10bit.Atmos.TrueHD.7.1-louco@PTHome
RELEASE DATE..............: 2021-01-21
RELEASE SiZE..............: 16.885 GB
SOURCE....................: 

.Media.Info
RUNTiME...................: 2 h 22 min
ViDEO CODEC...............: x265 Main 10@L5.1@High @ 12.54Mb/s
RESOLUTiON................: 1920x800
DiSPLAY ASPECT RATiO......: 2.400
FRAME RATE................: 23.976
BIT DEPTH.................: 10 bits
AUDiO CODEC...............: English MLP FBA 8 @ 4271.44Kb/s 
 
SUBTiTLES.................: S_HDMV/PGS : Chinese(chs&eng)
SUBTiTLES.................: S_HDMV/PGS : Chinese(cht&eng)
SUBTiTLES.................: S_HDMV/PGS : Chinese(chs)
SUBTiTLES.................: S_HDMV/PGS : Chinese(cht)
SUBTiTLES.................: S_HDMV/PGS : English(eng)


.x265.Info
x265 [info]: Main 10 profile, Level-5.1 (High tier)
x265 [info]: frame I:   1505, Avg QP:17.73  

# Step5 发布

In [1]:
# https://api.rhilip.info/ptgen.html

def get_bbcode(file, src="",logname="", title=None):
    print("Title:", file.replace("."," "))
#     由ZERO13@PTHome字幕组完成校对、调整。
    picbed='''
[url=https://sm.ms/image/PkKxy3XIiSgqF9E][img]https://i.loli.net/2021/01/25/PkKxy3XIiSgqF9E.jpg[/img][/url]
[url=https://sm.ms/image/DGFzvXgyKbiHAk6][img]https://i.loli.net/2021/01/25/DGFzvXgyKbiHAk6.jpg[/img][/url]
[url=https://sm.ms/image/gv97sCnXqeoywcK][img]https://i.loli.net/2021/01/25/gv97sCnXqeoywcK.jpg[/img][/url]
[url=https://sm.ms/image/xeb4HpnTQYAEJaK][img]https://i.loli.net/2021/01/25/xeb4HpnTQYAEJaK.jpg[/img][/url]
[url=https://sm.ms/image/1OLXTaMWNscAgEI][img]https://i.loli.net/2021/01/25/1OLXTaMWNscAgEI.jpg[/img][/url]
[url=https://sm.ms/image/CZyprLqXVl9kESU][img]https://i.loli.net/2021/01/25/CZyprLqXVl9kESU.jpg[/img][/url]
    '''
    if "REMUX" in file:
        picbed='''
[url=https://sm.ms/image/DXSGehZUtavp51n][img]https://i.loli.net/2021/01/21/DXSGehZUtavp51n.jpg[/img][/url]
[url=https://sm.ms/image/5tGfhgXBnrQCO1b][img]https://i.loli.net/2021/01/21/5tGfhgXBnrQCO1b.jpg[/img][/url]
[url=https://sm.ms/image/13UCdleYK7vAioP][img]https://i.loli.net/2021/01/21/13UCdleYK7vAioP.jpg[/img][/url]
[url=https://sm.ms/image/gFnQiyUTY4uH1Xj][img]https://i.loli.net/2021/01/21/gFnQiyUTY4uH1Xj.jpg[/img][/url]
[url=https://sm.ms/image/bpewl6C1vnYX5cF][img]https://i.loli.net/2021/01/21/bpewl6C1vnYX5cF.jpg[/img][/url]
[url=https://sm.ms/image/VpQHfztGyUBN2jX][img]https://i.loli.net/2021/01/21/VpQHfztGyUBN2jX.jpg[/img][/url]
    '''
    bbcode='''
[quote][size=5][color=RoyalBlue]原盘来自M-Team，diy来自xixihaha
字幕转载务必注意礼节，请保留原作者信息，谢绝二次提取修改！
在此感谢各位原作者及分享者！如有侵权请联系删除！[/color][/size][/quote]

[img]https://pic.imgdb.cn/item/600e6e7b3ffa7d37b3e7e87d.jpg[/img]

◎译　　名　92黑玫瑰对黑玫瑰 / 92 Legendary La Rose Noire / 黑蔷薇对黑蔷薇
◎片　　名　92黑玫瑰對黑玫瑰
◎年　　代　1992
◎产　　地　中国香港
◎类　　别　喜剧 / 动作 / 爱情 / 歌舞
◎语　　言　粤语
◎上映日期　1992-07-02
◎IMDb评分  6.8/10 from 315 users
◎IMDb链接  https://www.imdb.com/title/tt0103602
◎豆瓣评分　7.4/10 from 18366 users
◎豆瓣链接　https://movie.douban.com/subject/1299135/
◎片　　长　95分钟
◎导　　演　刘镇伟 Jeffrey Lau
◎编　　剧　刘镇伟 Jeffrey Lau
◎主　　演　梁家辉 Tony Leung
　　　　　  邵美琪 Maggie Siu
　　　　　  毛舜筠 Teresa Mo
　　　　　  黄韵诗 Wan-Si Wong
　　　　　  冯宝宝 Bo-Bo Fung
　　　　　  伍国健 Kwok-Kin Ng
　　　　　  孙名翠 Karen Suen
　　　　　  陈远佳 Yuen Kai Chan
　　　　　  司马华龙 Wah Lung Szema
　　　　　  叶进 Chun Yip
　　　　　  叶荣祖 Wing-Cho Yip
　　　　　  林祖辉 Thomas Lam
　　　　　  刘国昌 Lawrence Ah Mon
　　　　　  邓泰和 Tai Wo Tang


◎标　　签　香港 | 喜剧 | 梁家辉 | 刘镇伟 | 香港电影 | 毛舜筠 | 邵美琪 | 1992

◎简　　介  

　　黄蝴蝶（邵美琪 饰）是一个失意的编剧，几经波折，已经堕落到向闺蜜阿娟（毛舜筠 饰）探讨三级片的程度。胡蝶的邻居警官吕奇（梁家辉 饰）对她颇有好感，后者在英雄救美后意外得到了她家的钥匙，然而他的不请自来，却落得狼狈而逃。因为事业接连遭受重创，胡蝶决定在江边横刀自尽，然而怯懦 的她终于没有自裁，反而盲打误撞惊扰了一对偷情的鸳鸯。后者仓皇逃窜，却遗留了一些私人财物。为了物归原主，胡蝶和阿娟一同登门，却无意中卷入了一场黑吃黑的血案。侥幸逃生的胡蝶临走前，还多事地留下了黑玫瑰的字条，而阿娟却因为留下了手印而被警方对号入座。结果，黑玫瑰成为了全城热议的话题，而这也引起了黑玫瑰掌门人（冯宝宝 饰）与师姐（黄韵诗 饰）的注意，于是一场剪不断理还乱的情场争夺战开始上演了……

◎获奖情况  

　　第12届香港电影金像奖(1993)
　　最佳电影(提名) 浮乐莲
　　最佳导演(提名) 刘镇伟
　　最佳编剧(提名) 刘镇伟
　　最佳男主角 梁家辉
　　最佳女配角 冯宝宝
　　最佳女配角(提名) 毛舜筠 / 黄韵诗
　　最佳服装造型设计(提名) Joseph Chan
　　最佳原创电影音乐(提名) 卢冠廷

[img]https://images2.imgbox.com/04/6b/Ggp5ReQb_o.png[/img]

'''+f'{generate_videoinfo(file,logname,src=src)}\n{picbed}\n{get_mediainfo(file)}'
    pyperclip.copy(bbcode)
SRC=r"Legendary La Rose Noire 1992 REMUX Blu-ray 1080p AVC TrueHD 5.1-DIY@xixihaha".replace(" ",".")

In [4]:
from utils import *
REMUX=MOVIE
get_bbcode(REMUX,src=SRC)

Title: Legendary La Rose Noire 1992 Blu-ray 1080p REMUX TrueHD 5 1-louco@PTHome mkv
[quote][font=Lucida Console]
.Release.Info
ENCODER...................: PTH
RELEASE NAME..............: Legendary.La.Rose.Noire.1992.Blu-ray.1080p.REMUX.TrueHD.5.1-louco@PTHome
RELEASE DATE..............: 2021-01-25
RELEASE SiZE..............: 19.005 GB
SOURCE....................: Legendary.La.Rose.Noire.1992.REMUX.Blu-ray.1080p.AVC.TrueHD.5.1-DIY@xixihaha

.Media.Info
RUNTiME...................: 1 h 34 min
ViDEO CODEC...............: ? High@L4.1 @ 25.43Mb/s
RESOLUTiON................: 1920x1080
DiSPLAY ASPECT RATiO......: 1.778
FRAME RATE................: 24.000
BIT DEPTH.................: 8 bits
AUDiO CODEC...............: Chinese AC-3 2 @ 384.0Kb/s 
AUDiO CODEC...............: Chinese MLP FBA 5.1 @ 2745.61Kb/s 
 
SUBTiTLES.................: S_HDMV/PGS : Chinese(chs)
SUBTiTLES.................: S_HDMV/PGS : Chinese(cht)
SUBTiTLES.................: S_HDMV/PGS : English(eng)


[/quote]
Copy Mediainfo


In [13]:
def generate_videoinfo(filename, logname="", src=""):
    media_info = MediaInfo.parse(filename)
    for t in media_info.video_tracks:
        props=nullabledict(t.to_data())
#         for i in props.items(): print(i)
        video_codec=f"{props['encoded_library_name']} {props['format_profile']} @ {str(round(props['bit_rate'] / (1000 * 1000), 2))}Mb/s"
        runtime=props['other_duration'][0]
        resolution=f"{props['sampled_width']}x{props['sampled_height']}"
        display_aspect_ratio=props['display_aspect_ratio']
        frame_rate=props["frame_rate"]
        bit_depth=props["other_bit_depth"][0]
    audio_codec=[]
    for t in media_info.audio_tracks:
        props=nullabledict(t.to_data())
#         for i in t.to_data().items(): print(i)
        chns=props['channel_s']
        if chns==6: chns="5.1"
        dtsma=""
        if props['commercial_name']=='DTS-HD Master Audio':dtsma="-HD MA"
        audio_codec.append(f"{props['other_language'][0]} {props['format']}{dtsma} {chns}\
 @ {str(round(props['bit_rate'] / (1000), 2))}Kb/s")
    audioinfo=""    
    for ac in audio_codec:
        audioinfo+=f'AUDiO CODEC...............: {ac} \n'
    chscnt=0
    subtitledesc=''
    for t in media_info.text_tracks:
        props=nullabledict(t.to_data(),default=["Chinese"])
    #     for i in props.items(): print(i)
        if 'title' in props:
            title=props['title']
        else: title=props['language']
        
        if title=='zh': 
            title='chs';chscnt+=1
            if chscnt>2: title='cht'
            if chscnt%2==0: title+='&eng'
        subtitlefmt=f"{props['codec_id']} : {props['other_language'][0]}({title})"
        subtitledesc+=f"SUBTiTLES.................: {subtitlefmt}\n"
    NEWMOVIE_size=str(round(os.path.getsize(filename) / (1024 * 1024 * 1024), 3)) + ' GB'
    MOVIE_filename="".join(re.findall(r'([^\\]*$)', src))
    RELEASE_filename="".join(re.findall(r"([^\\]*$)", filename.replace(".mkv","")))
    
    transcodelog=""
    if len(logname)>0:
        transcodelog=f'.{logname.replace(".log","")}.Info\n'
        with open(logname, encoding="utf-8") as f:
            lines=f.readlines()
            for line in lines:
                if ('[libx' in line or '[info]' in line) and 'profile' not in line:
                    if 'Avg QP' in line or 'profile' in line or 'rames:' in line:
                        transcodelog+=line
    release_info=f'''[quote][font=Lucida Console]
.Release.Info
ENCODER...................: PTH
RELEASE NAME..............: {RELEASE_filename}
RELEASE DATE..............: {datetime.today().strftime('%Y-%m-%d')}
RELEASE SiZE..............: {NEWMOVIE_size}
SOURCE....................: {MOVIE_filename.replace(".mkv","")}

.Media.Info
RUNTiME...................: {runtime}
ViDEO CODEC...............: {video_codec}
RESOLUTiON................: {resolution}
DiSPLAY ASPECT RATiO......: {display_aspect_ratio}
FRAME RATE................: {frame_rate}
BIT DEPTH.................: {bit_depth}
{audioinfo} 
{subtitledesc}

{transcodelog}[/quote]'''
    print(release_info)
    pyperclip.copy(release_info)
    return release_info

In [11]:
get_bbcode(x264_fullpath,src=SRC,logname="x264.log")

Title: I:\Users\heber\Music\Legendary La Rose Noire 1992 Blu-ray 1080p x264 TrueHD 5 1-louco@PTHome\Legendary La Rose Noire 1992 Blu-ray 1080p x264 TrueHD 5 1-louco@PTHome mkv
[quote][font=Lucida Console]
.Release.Info
ENCODER...................: PTH
RELEASE NAME..............: Legendary.La.Rose.Noire.1992.Blu-ray.1080p.x264.TrueHD.5.1-louco@PTHome
RELEASE DATE..............: 2021-01-25
RELEASE SiZE..............: 10.084 GB
SOURCE....................: Legendary.La.Rose.Noire.1992.REMUX.Blu-ray.1080p.AVC.TrueHD.5.1-DIY@xixihaha

.Media.Info
RUNTiME...................: 1 h 34 min
ViDEO CODEC...............: x264 High@L4.1 @ 11.97Mb/s
RESOLUTiON................: 1920x1080
DiSPLAY ASPECT RATiO......: 1.778
FRAME RATE................: 24.000
BIT DEPTH.................: 8 bits
AUDiO CODEC...............: Chinese AC-3 2 @ 384.0Kb/s 
AUDiO CODEC...............: Chinese MLP FBA 5.1 @ 2745.61Kb/s 
 
SUBTiTLES.................: S_HDMV/PGS : Chinese(chs)
SUBTiTLES.................: S_HDMV/PGS : Ch

In [14]:
get_bbcode(x265_fullpath,src=SRC,logname="x265.log")

Title: I:\Users\heber\Music\Legendary La Rose Noire 1992 Blu-ray 1080p x265 10bit TrueHD 5 1-louco@PTHome\Legendary La Rose Noire 1992 Blu-ray 1080p x265 10bit TrueHD 5 1-louco@PTHome mkv
[quote][font=Lucida Console]
.Release.Info
ENCODER...................: PTH
RELEASE NAME..............: Legendary.La.Rose.Noire.1992.Blu-ray.1080p.x265.10bit.TrueHD.5.1-louco@PTHome
RELEASE DATE..............: 2021-01-25
RELEASE SiZE..............: 7.717 GB
SOURCE....................: Legendary.La.Rose.Noire.1992.REMUX.Blu-ray.1080p.AVC.TrueHD.5.1-DIY@xixihaha

.Media.Info
RUNTiME...................: 1 h 34 min
ViDEO CODEC...............: x265 Main@L5.1@High @ 8.4Mb/s
RESOLUTiON................: 1920x1080
DiSPLAY ASPECT RATiO......: 1.778
FRAME RATE................: 24.000
BIT DEPTH.................: 8 bits
AUDiO CODEC...............: Chinese AC-3 2 @ 384.0Kb/s 
AUDiO CODEC...............: Chinese MLP FBA 5.1 @ 2745.61Kb/s 
 
SUBTiTLES.................: S_HDMV/PGS : Chinese(chs)
SUBTiTLES..............

# 备忘录